<a href="https://colab.research.google.com/github/sailTo/essay/blob/master/V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import SGDClassifier, LogisticRegression

dfoff = pd.read_csv('/content/drive/Shareddrives/ubox/ccf_offline_stage1_train.csv')
dftest = pd.read_csv('/content/drive/Shareddrives/ubox/ccf_offline_stage1_test_revised.csv')
dfon = pd.read_csv('/content/drive/Shareddrives/ubox/ccf_online_stage1_train.csv')
print('All data are read.')

All data are read.


In [ ]:
def float_to_datetime(row):
    if row != 'nan': return date(int(row[0:4]), int(row[4:6]), int(row[6:8]))
    else: return np.nan

dfoff['Date_received'] = dfoff['Date_received'].astype(str).apply(float_to_datetime)
dftest['Date_received'] = dftest['Date_received'].astype(str).apply(float_to_datetime)
dfoff['Date'] = dfoff['Date'].astype(str).apply(float_to_datetime)


In [ ]:
def get_offline_features(X, offline):
    # X = X[:1000]

    print(len(X), len(X.columns))

    coupon_id_notnull = offline[offline.Coupon_id.notnull()]

    temp = coupon_id_notnull
    coupon_consume = temp[temp.Date.notnull()]
    coupon_no_consume = temp[temp.Date.isnull()]

    user_coupon_consume = coupon_consume.groupby('User_id')

        # X['weekday'] = X.Date_received.dt.weekday
        # X['day'] = X.Date_received.dt.day

        # # 距离优惠券消费次数
        # temp = coupon_consume.groupby('Distance').size().reset_index(name='distance_0')
        # X = pd.merge(X, temp, how='left', on='Distance')
        #
        # # 距离优惠券不消费次数
        # temp = coupon_no_consume.groupby('Distance').size().reset_index(name='distance_1')
        # X = pd.merge(X, temp, how='left', on='Distance')
        #
        # # 距离优惠券领取次数
        # X['distance_2'] = X.distance_0 + X.distance_1
        #
        # # 距离优惠券消费率
        # X['distance_3'] = X.distance_0 / X.distance_2

        # temp = coupon_consume[coupon_consume.Distance != 11].groupby('Distance').size()
        # temp['d4'] = temp.Distance.sum() / len(temp)
        # X = pd.merge(X, temp, how='left', on='Distance')

    '''user features'''

        # # 用户优惠券消费次数
        # temp = user_coupon_consume.size().reset_index(name='u2')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户优惠券不消费次数
        # temp = coupon_no_consume.groupby('User_id').size().reset_index(name='u3')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户优惠券消费与不消费次数比值
        # X['u19'] = X.u2 / X.u3

        # # 用户领取优惠券次数
        # X['u1'] = X.u2.fillna(0) + X.u3.fillna(0)

        # # 用户优惠券使用率
        # X['u4'] = X.u2 / X.u1

    # 用户普通消费次数
    date_received_isnull_date_notnull = offline[offline.Date_received.isnull() & offline.Date.notnull()]

    temp = date_received_isnull_date_notnull.groupby('User_id').size().reset_index(name='u5')
    X = pd.merge(X, temp, how='left', on='User_id')
    print(X.columns)
    X.u5 = X.u5.fillna(0)

            # # 用户优惠券消费和普通消费次数
            # X['u25'] = X.u2 + X.u5

            # # 用户使用优惠券消费占比
            # X['u20'] = X.u2 / X.u25

        # # 用户所有消费平均间隔
        # temp = offline[offline.Date.notnull()]
        # temp = pd.merge(temp, temp.groupby('User_id').Date.max().reset_index(name='_max'))
        # temp = pd.merge(temp, temp.groupby('User_id').Date.min().reset_index(name='_min'))
        # temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='_len'))
        # temp['u6'] = (temp._max - temp._min).dt.days / (temp._len - 1)
        # temp.drop_duplicates('User_id', inplace=True)
        # X = pd.merge(X, temp[['User_id', 'u6']], how='left', on='User_id')

        # # 用户普通消费平均间隔
        # temp = date_received_isnull_date_notnull
        # temp = pd.merge(temp, temp.groupby('User_id').Date.max().reset_index(name='_max'))
        # temp = pd.merge(temp, temp.groupby('User_id').Date.min().reset_index(name='_min'))
        # temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='_len'))
        # temp['u26'] = (temp._max - temp._min).dt.days / (temp._len - 1)
        # temp.drop_duplicates('User_id', inplace=True)
        # X = pd.merge(X, temp[['User_id', 'u26']], how='left', on='User_id')

        # # 用户优惠券消费平均间隔
        # temp = pd.merge(coupon_consume, user_coupon_consume.Date.max().reset_index(name='_max'))
        # temp = pd.merge(temp, temp.groupby('User_id').Date.min().reset_index(name='_min'))
        # temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='_len'))
        # temp['u7'] = (temp._max - temp._min).dt.days / (temp._len - 1)
        # temp.drop_duplicates('User_id', inplace=True)
        # X = pd.merge(X, temp[['User_id', 'u7']], how='left', on='User_id')

        # # 用户15天内平均会普通消费几次
        # X['u8'] = X.u6 / 15

        # # 用户15天内平均会优惠券消费几次
        # X['u9'] = X.u7 / 15

        # # 用户优惠券领取到使用平均间隔
        # temp = coupon_consume.copy()
        # temp['days'] = (temp.Date - temp.Date_received).dt.days
        # temp = (temp.groupby('User_id').days.sum() / temp.groupby('User_id').size()).reset_index(name='u10')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户在15天内使用掉优惠券的值大小
        # X['u11'] = X.u10 / 15

        # # 领取优惠券到使用优惠券间隔小于15天的次数
        # temp = coupon_consume.copy()
        # temp['days'] = (temp.Date - temp.Date_received).dt.days
        # temp = temp[temp.days <= 15]
        # temp = temp.groupby('User_id').size().reset_index(name='u21')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户15天使用掉优惠券的次数除以使用优惠券的次数
        # X['u22'] = X.u21 / X.u2

        # # 用户15天使用掉优惠券的次数除以领取优惠券未消费的次数
        # X['u23'] = X.u21 / X.u3

        # # 用户15天使用掉优惠券的次数除以领取优惠券的总次数
        # X['u24'] = X.u21 / X.u1

        # # 用户核销优惠券的平均折率
        # temp = user_coupon_consume.discount.mean().reset_index(name='u45')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户核销优惠券的最低折率
        # temp = user_coupon_consume.discount.min().reset_index(name='u27')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户核销优惠券的最高消费折率
        # temp = user_coupon_consume.discount.max().reset_index(name='u28')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户核销过的不同优惠券数量
        # temp = coupon_consume.groupby(['User_id', 'Coupon_id']).size()
        # temp = temp.groupby('User_id').size().reset_index(name='u32')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户领取所有不同优惠券数量
        # date_received_notnull = offline[offline.Date_received.notnull()]
        # temp = date_received_notnull
        # temp = temp.groupby(['User_id', 'Coupon_id']).size().reset_index(name='u47')
        # X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id'])

        # # 用户核销过的不同优惠券数量占所有不同优惠券的比重
        # X['u33'] = X.u32 / X.u47

        # # 用户平均每种优惠券核销多少张
        # X['u34'] = X.u2 / X.u47

    # 核销优惠券用户-商家平均距离 fill_avg
    temp = offline[offline.Coupon_id.notnull() & offline.Date.notnull() & offline.Distance.notnull()]
    temp = temp.groupby('User_id').Distance
    temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='User_id')
    temp['u35'] = temp.y / temp.x
    temp = temp[['User_id', 'u35']]
    X = pd.merge(X, temp, how='left', on='User_id')
    X.u35=X.u35.fillna(sum(X.u35)/sum(X.u35.notnull()))

    # 用户核销优惠券中的最小用户-商家距离
    temp = coupon_consume[coupon_consume.Distance.notnull()]
    temp = temp.groupby('User_id').Distance.min().reset_index(name='u36')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.u35=X.u36.fillna(sum(X.u36)/sum(X.u36.notnull()))

    # 用户核销优惠券中的最大用户-商家距离
    temp = coupon_consume[coupon_consume.Distance.notnull()]
    temp = temp.groupby('User_id').Distance.max().reset_index(name='u37')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.u35=X.u35.fillna(sum(X.u37)/sum(X.u37.notnull()))

    # 优惠券类型
    discount_types = [
        '0.2', '0.5', '0.6', '0.7', '0.75', '0.8', '0.85', '0.9', '0.95', '30:20', '50:30', '10:5',
        '20:10', '100:50', '200:100', '50:20', '30:10', '150:50', '100:30', '20:5', '200:50', '5:1',
        '50:10', '100:20', '150:30', '30:5', '300:50', '200:30', '150:20', '10:1', '50:5', '100:10',
        '200:20', '300:30', '150:10', '300:20', '500:30', '20:1', '100:5', '200:10', '30:1', '150:5',
        '300:10', '200:5', '50:1', '100:1',
    ]
    X['discount_type'] = -1
    for k, v in enumerate(discount_types):
        X.loc[X.Discount_rate == v, 'discount_type'] = k
    
            # # 不同优惠券领取次数
            # temp = offline.groupby(['User_id', 'Discount_rate']).size().reset_index(name='u41')
            # X = pd.merge(X, temp, how='left', on=['User_id', 'Discount_rate'])

            # # 不同优惠券使用次数
            # temp = coupon_consume.groupby(['User_id', 'Discount_rate']).size().reset_index(name='u42')
            # X = pd.merge(X, temp, how='left', on=['User_id', 'Discount_rate'])

            # # 不同优惠券不使用次数
            # temp = coupon_no_consume.groupby(['User_id', 'Discount_rate']).size().reset_index(name='u43')
            # X = pd.merge(X, temp, how='left', on=['User_id', 'Discount_rate'])

            # # 不同打折优惠券使用率
            # X['u44'] = X.u42 / X.u41

            # # 满减类型优惠券领取次数
            # temp = offline[offline.Discount_rate.str.contains(':') == True]
            # temp = temp.groupby('User_id').size().reset_index(name='u48')
            # X = pd.merge(X, temp, how='left', on='User_id')

            # # 打折类型优惠券领取次数
            # temp = offline[offline.Discount_rate.str.contains('\.') == True]
            # temp = temp.groupby('User_id').size().reset_index(name='u49')
            # X = pd.merge(X, temp, how='left', on='User_id')

    '''offline merchant features'''

    # 商家销售次数
    temp = offline[offline.Date.notnull()].groupby('Merchant_id').size().reset_index(name='m0')
    X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商家优惠券核销次数
    temp = coupon_consume.groupby('Merchant_id').size().reset_index(name='m1')
    X = pd.merge(X, temp, how='left', on='Merchant_id')
    X.m1 = X.m1.fillna(0)

        # # 商家普通销售次数
        # X['m2'] = X.m0.fillna(0) - X.m1.fillna(0)

        # # 商家优惠券被领取次数
        # temp = date_received_notnull.groupby('Merchant_id').size().reset_index(name='m3')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家优惠券核销率
        # X['m4'] = X.m1 / X.m3

        # # 商家优惠券不核销次数
        # temp = coupon_no_consume.groupby('Merchant_id').size().reset_index(name='m7')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家在数据集中出现的次数
        # temp = offline.groupby('Merchant_id').size().reset_index(name='m16')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

    # 商户当天优惠券领取次数
    temp = X[X.Date_received.notnull()]
    temp = temp.groupby(['Merchant_id', 'Date_received']).size().reset_index(name='m5')
    X = pd.merge(X, temp, how='left', on=['Merchant_id', 'Date_received'])
    X.m5 = X.m5.fillna(0)

        # # 商家当天优惠券领取人数
        # temp = X[X.Date_received.notnull()]
        # temp = temp.groupby(['User_id', 'Merchant_id', 'Date_received']).size()
        # temp = temp.groupby(['Merchant_id', 'Date_received']).size().reset_index(name='m6')
        # X = pd.merge(X, temp, how='left', on=['Merchant_id', 'Date_received'])

        # # 商家核销优惠券的平均折率
        # temp = coupon_consume.groupby('Merchant_id').discount.mean().reset_index(name='m8')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家优惠券核销的最小消费折率
        # temp = coupon_consume.groupby('Merchant_id').discount.max().reset_index(name='m9')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家优惠券核销的最大消费折率
        # temp = coupon_consume.groupby('Merchant_id').discount.min().reset_index(name='m10')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家优惠券核销不同的用户数量
        # temp = coupon_consume.groupby(['Merchant_id', 'User_id']).size()
        # temp = temp.groupby('Merchant_id').size().reset_index(name='m11')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家优惠券领取不同的用户数量
        # temp = date_received_notnull.groupby(['Merchant_id', 'User_id']).size()
        # temp = temp.groupby('Merchant_id').size().reset_index(name='m12')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 核销商家优惠券的不同用户数量其占领取不同的用户比重
        # X['m13'] = X.m11 / X.m12

        # # 商家优惠券平均每个用户核销多少张
        # X['m14'] = X.m1 / X.m12

        # # 商家被核销过的不同优惠券数量
        # temp = coupon_consume.groupby(['Merchant_id', 'Coupon_id']).size()
        # temp = temp.groupby('Merchant_id').size().reset_index(name='m15')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家领取过的不同优惠券数量的比重
        # temp = date_received_notnull.groupby(['Merchant_id', 'Coupon_id']).size()
        # temp = temp.groupby('Merchant_id').count().reset_index(name='m18')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家被核销过的不同优惠券数量占所有领取过的不同优惠券数量的比重
        # X['m19'] = X.m15 / X.m18

    # 商家被核销优惠券的平均时间
    temp = pd.merge(coupon_consume, coupon_consume.groupby('Merchant_id').Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('Merchant_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('Merchant_id').size().reset_index(name='_len'))
    temp['m20'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('Merchant_id', inplace=True)
    X = pd.merge(X, temp[['Merchant_id', 'm20']], how='left', on='Merchant_id')
    X.m20 = X.m20.fillna(15)

    # 商家被核销优惠券中的用户平均距离
    temp = coupon_consume[coupon_consume.Distance.notnull()].groupby('Merchant_id').Distance
    temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='Merchant_id')
    temp['m21'] = temp.y / temp.x
    temp = temp[['Merchant_id', 'm21']]
    X = pd.merge(X, temp, how='left', on='Merchant_id')
    X.m21=X.m21.fillna(10)

        # # 商家被核销优惠券中的用户最小距离
        # temp = coupon_consume[coupon_consume.Distance.notnull()]
        # temp = temp.groupby('Merchant_id').Distance.min().reset_index(name='m22')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家被核销优惠券中的用户最大距离
        # temp = coupon_consume[coupon_consume.Distance.notnull()]
        # temp = temp.groupby('Merchant_id').Distance.max().reset_index(name='m23')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # 商家普通消费用户的平均距离
        # temp = offline[pd.isnull(offline.Date_received) & pd.notnull(offline.Date) & pd.notnull(offline.Distance)]
        # temp = temp.groupby('Merchant_id').Distance
        # temp = pd.merge(temp.count().reset_index(name='x'), temp.sum().reset_index(name='y'), on='Merchant_id')
        # temp['m24'] = temp.y / temp.x
        # temp = temp[['Merchant_id', 'm24']]
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

    """offline coupon features"""

    # 每种优惠券领取次数
    temp = coupon_id_notnull.groupby('Coupon_id').size().reset_index(name='c1')
    X = pd.merge(X, temp, how='left', on='Coupon_id')
    X.c1=X.c1.fillna(0)

    # 每种优惠券使用次数
    temp = coupon_consume.groupby('Coupon_id').size().reset_index(name='c2')
    X = pd.merge(X, temp, how='left', on='Coupon_id')
    X.c2 = X.c2.fillna(0)

    # 优惠券使用率
    X['c3'] = X.c2 / X.c1
    X.drop(['c1','c2'],axis=1)
    X.u5 = X.u5.fillna(sum(X.u5)/sum(X.u5.notnull()))

    # # 优惠券没有使用次数
    # X['c4'] = X.c1 - X.c2
    

    # # 优惠券当天领取次数
    # temp = X.groupby(['Coupon_id', 'Date_received']).size().reset_index(name='c5')
    # X = pd.merge(X, temp, how='left', on=['Coupon_id', 'Date_received'])

    # 优惠券类型(直接优惠为0, 满减为1)
    X['c6'] = 0 
    X.loc[X.Discount_rate.str.contains(':') == True, 'c6'] = 1

        # # 不同打折优惠券领取次数
        # temp = offline.groupby('Discount_rate').size().reset_index(name='c8')
        # X = pd.merge(X, temp, how='left', on='Discount_rate')

        # # 不同打折优惠券使用次数
        # temp = coupon_consume.groupby('Discount_rate').size().reset_index(name='c9')
        # X = pd.merge(X, temp, how='left', on='Discount_rate')

        # # 不同打折优惠券不使用次数
        # temp = coupon_no_consume.groupby('Discount_rate').size().reset_index(name='c10')
        # X = pd.merge(X, temp, how='left', on='Discount_rate')

        # # 不同打折优惠券使用率
        # X['c11'] = X.c9 / X.c8

    # 每种优惠券核销的平均时间
    temp = pd.merge(coupon_consume, coupon_consume.groupby('Coupon_id').Date.max().reset_index(name='_max'))
    temp = pd.merge(temp, temp.groupby('Coupon_id').Date.min().reset_index(name='_min'))
    temp = pd.merge(temp, temp.groupby('Coupon_id').size().reset_index(name='_len'))
    temp['c12'] = (temp._max - temp._min).dt.days / (temp._len - 1)
    temp.drop_duplicates('Coupon_id', inplace=True)
    X = pd.merge(X, temp[['Coupon_id', 'c12']], how='left', on='Coupon_id')
    X.c12=X.c12.fillna(sum(X.c12)/sum(X.c12.notnull()))

    '''user merchant feature'''

    # 用户领取商家的优惠券次数
    temp = coupon_id_notnull
    temp = temp.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um1')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])
    X.um1 = X.um1.fillna(0)

    # 用户领取商家的优惠券后核销次数
    temp = coupon_consume.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um3')
    X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])
    X.um3 = X.um3.fillna(0)

    # 用户领取商家的优惠券后核销率
    X['um4'] = X.um3 / X.um1
    X.um4=X.um4.fillna(0)
    X = X.drop(['um1','um3'],axis=1)

        # # 用户对每个商家的不核销次数占用户总的不核销次数的比重
        # temp = coupon_no_consume.groupby('User_id').size().reset_index(name='temp')
        # X = pd.merge(X, temp, how='left', on='User_id')
        # X['um5'] = X.um2 / X.temp
        # X.drop(columns='temp', inplace=True)

        # # 用户在商店总共消费过几次
        # temp = offline[offline.Date.notnull()].groupby(['User_id', 'Merchant_id']).size().reset_index(name='um6')
        # X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

        # # 用户在商店普通消费次数
        # temp = date_received_isnull_date_notnull
        # temp = temp.groupby(['User_id', 'Merchant_id']).size().reset_index(name='um7')
        # X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

        # # 用户当天在每个商家领取的优惠券次数
        # temp = date_received_notnull
        # temp = temp.groupby(['User_id', 'Merchant_id', 'Date_received']).size().reset_index(name='um8')
        # X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id', 'Date_received'])

        # # 用户领取优惠券不同商家数量
        # temp = coupon_id_notnull
        # temp = temp.groupby(['User_id', 'Merchant_id']).size().reset_index()
        # temp = temp.groupby('User_id').size().reset_index(name='um9')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户核销优惠券不同商家数量
        # temp = coupon_consume.groupby(['User_id', 'Merchant_id']).size()
        # temp = temp.groupby('User_id').size().reset_index(name='um10')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户核销过优惠券的不同商家数量占所有不同商家的比重
        # X['um11'] = X.um10 / X.um9

        # # 用户平均核销每个商家多少张优惠券
        # X['um12'] = X.u2 / X.um9

    '''other feature'''

    # 用户领取优惠券次数
    temp = X.groupby('User_id').size().reset_index(name='o1')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.o1 = X.o1.fillna(0)

        # # 用户不同优惠券领取次数
        # temp = X.groupby(['User_id', 'Coupon_id']).size().reset_index(name='o2')
        # X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id'])

        # # 用户此次之后/前领取的优惠券次数
        # X['o3'] = 1
        # X['o3'] = X.sort_values(by=['User_id', 'Date_received']).groupby('User_id').o3.cumsum() - 1
        # X['o4'] = 1
        # X['o4'] = X.sort_values(by=['User_id', 'Date_received'], ascending=False).groupby('User_id').o4.cumsum() - 1

        # # 用户此次之后/前领取的每种优惠券次数
        # X['o5'] = 1
        # temp = X.sort_values(by=['User_id', 'Coupon_id', 'Date_received'])
        # X['o5'] = temp.groupby('User_id').o5.cumsum() - 1
        # X['o6'] = 1
        # temp = X.sort_values(by=['User_id', 'Coupon_id', 'Date_received'], ascending=False)
        # X['o6'] = temp.groupby('User_id').o6.cumsum() - 1

        # 用户领取优惠券平均时间间隔
        # temp = pd.merge(X, X.groupby('User_id').Date_received.max().reset_index(name='_max'))
        # temp = pd.merge(temp, temp.groupby('User_id').Date_received.min().reset_index(name='_min'))
        # temp = pd.merge(temp, temp.groupby('User_id').size().reset_index(name='_len'))
        # temp['o7'] = (temp._max - temp._min).dt.days / (temp._len - 1)
        # temp.drop_duplicates('User_id', inplace=True)
        # X = pd.merge(X, temp[['User_id', 'o7']], how='left', on='User_id')

        # # 用户领取不同商家的优惠券次数
        # temp = X.groupby(['User_id', 'Merchant_id']).size().reset_index(name='o8')
        # X = pd.merge(X, temp, how='left', on=['User_id', 'Merchant_id'])

        # # 用户领取的不同商家数
        # temp = X.groupby(['User_id', 'Merchant_id']).size()
        # temp = temp.groupby('User_id').size().reset_index(name='o9')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 用户当天领取的优惠券次数
        # temp = X.groupby(['User_id', 'Date_received']).size().reset_index(name='o10')
        # X = pd.merge(X, temp, how='left', on=['User_id', 'Date_received'])

        # # 用户当天不同优惠券领取次数
        # temp = X.groupby(['User_id', 'Coupon_id', 'Date_received']).size().reset_index(name='o11')
        # X = pd.merge(X, temp, how='left', on=['User_id', 'Coupon_id', 'Date_received'])

        # # 用户领取优惠券类别数
        # temp = X.groupby(['User_id', 'Coupon_id']).size()
        # temp = temp.groupby('User_id').size().reset_index(name='o12')
        # X = pd.merge(X, temp, how='left', on='User_id')

        # # 商家被领取的优惠券次数
        # temp = X.groupby('Merchant_id').size().reset_index(name='o13')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家优惠券的种类数
        # temp = X.groupby(['Merchant_id', 'Coupon_id']).size().reset_index(name='o14')
        # X = pd.merge(X, temp, how='left', on=['Merchant_id', 'Coupon_id'])

        # # 商家被领取优惠券不同用户数
        # temp = X.groupby(['Merchant_id', 'User_id']).size()
        # temp = temp.groupby('Merchant_id').size().reset_index(name='o15')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 商家优惠券所有种类数
        # temp = X.groupby(['Merchant_id', 'Coupon_id']).size()
        # temp = temp.groupby('Merchant_id').size().reset_index(name='o16')
        # X = pd.merge(X, temp, how='left', on='Merchant_id')

        # # 用户领取优惠券的时间间隔
        # temp = X.sort_values(by=['User_id', 'Date_received']).groupby('User_id')
        # X['o17'] = temp.Date_received.diff().dt.days
        # X['o18'] = temp.Date_received.diff(-1).dt.days.abs()

    print(len(X), len(X.columns))

    return X


def get_online_features(X, online):
    # temp = online[online.Coupon_id == online.Coupon_id]
    # coupon_consume = temp[temp.Date == temp.Date]
    # coupon_no_consume = temp[temp.Date != temp.Date]

    # 用户操作次数
    temp = online.groupby('User_id').size().reset_index(name='on_u1')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u1 = X.on_u1.fillna(0)

    # 用户点击次数
    temp = online[online.Action == 0].groupby('User_id').size().reset_index(name='on_u2')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u2 = X.on_u2.fillna(0)

    # 用户点击率
    X['on_u3'] = X.on_u2 / X.on_u1
    X.on_u3 = X.on_u3.fillna(0)

    # 用户购买次数
    temp = online[online.Action == 1].groupby('User_id').size().reset_index(name='on_u4')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u4 = X.on_u4.fillna(0)

    # 用户线上购买率
    X['on_u5'] = X.on_u4 / X.on_u1
    X.on_u5 = X.on_u5.fillna(0)

    # 用户领取优惠券次数
    temp = online[online.Date_received.notnull()].groupby('User_id').size().reset_index(name='on_u6')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u6 = X.on_u6.fillna(0)

    # 用户优惠券领取率
    X['on_u7'] = X.on_u6 / X.on_u1
    X.on_u7 = X.on_u7.fillna(0)

    # 用户优惠券消费次数
    temp = online[online.Date_received.notnull() & online.Date.notnull()]
    temp = temp.groupby('User_id').size().reset_index(name='on_u9')
    X = pd.merge(X, temp, how='left', on='User_id')
    X.on_u9 = X.on_u9.fillna(0)

        # # 用户优惠券不消费次数
        # temp = online[online.Date_received.notnull() & online.Date.isnull()]
        # temp = temp.groupby('User_id').size().reset_index(name='on_u8')
        # X = pd.merge(X, temp, how='left', on='User_id')

    # 用户优惠券核销率
    X['on_u10'] = X.on_u9 / X.on_u6
    X = X.drop(['on_u1','on_u2','on_u4','on_u6','on_u9'],axis=1)
    X.on_u10 = X.on_u10.fillna(0)

        # # 用户线下不消费次数占线上线下总的不消费次数的比重
        # X['on_u11'] = X.u3 / (X.on_u8.fillna(0) + X.u3.fillna(0))

        # # 用户线下的优惠券核销次数占线上线下总的优惠券核销次数的比重
        # X['on_u12'] = X.u2 / (X.on_u9.fillna(0) + X.u2.fillna(0))

        # # 用户线下领取的记录数量占总的记录数量的比重
        # X['on_u13'] = X.u1 / (X.on_u6.fillna(0) + X.u1.fillna(0))

        # # 消费优惠券的平均折率
        # temp = coupon_consume.groupby('User_id').discount.mean().reset_index(name='ou14')
        # X = pd.merge(X, temp, how='left', on='User_id')
        #
        # # 用户核销优惠券的最低消费折率
        # temp = coupon_consume.groupby('User_id').discount.min().reset_index(name='ou15')
        # X = pd.merge(X, temp, how='left', on='User_id')
        #
        # # 用户核销优惠券的最高消费折率
        # temp = coupon_consume.groupby('User_id').discount.max().reset_index(name='ou16')
        # X = pd.merge(X, temp, how='left', on='User_id')
        #
        # # 不同打折优惠券领取次数
        # temp = online.groupby('Discount_rate').size().reset_index(name='oc1')
        # X = pd.merge(X, temp, how='left', on='Discount_rate')
        #
        # # 不同打折优惠券使用次数
        # temp = coupon_consume.groupby('Discount_rate').size().reset_index(name='oc2')
        # X = pd.merge(X, temp, how='left', on='Discount_rate')
        #
        # # 不同打折优惠券不使用次数
        # temp = coupon_no_consume.groupby('Discount_rate').size().reset_index(name='oc3')
        # X = pd.merge(X, temp, how='left', on='Discount_rate')
        #
        # # 不同打折优惠券使用率
        # X['oc4'] = X.oc2 / X.oc1

    print(len(X), len(X.columns))
    print('----------')

    return X

In [ ]:
# -1
    # def getWeekday(row):
    #     if row == 'nan':
    #         return np.nan
    #     else:
    #         return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

    # dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
    # dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)

    # # weekday_type :  周六和周日为1，其他为0
    # dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )
    # dftest['weekday_type'] = dftest['weekday'].apply(lambda x : 1 if x in [6,7] else 0 )

    # # change weekday to one-hot encoding 
    # weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
    # tmpdf = pd.get_dummies(dfoff['weekday'].replace('nan', np.nan))
    # tmpdf.columns = weekdaycols
    # dfoff[weekdaycols] = tmpdf

    # tmpdf = pd.get_dummies(dftest['weekday'].replace('nan', np.nan))
    # tmpdf.columns = weekdaycols
    # dftest[weekdaycols] = tmpdf

dftest = get_offline_features(dftest,dfoff)
dftest = get_online_features(dftest,dfon)

113640 6
Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'u5'],
      dtype='object')
113640 23
113640 27
----------


In [ ]:
# 0
    # # 1. 将满xx减yy类型(`xx:yy`)的券变成折扣率 : `1 - yy/xx`，同时建立折扣券相关的特征 `discount_rate, discount_man, discount_jian, discount_type`
    # # 2. 将距离 `str` 转为 `int`
    # # convert Discount_rate and Distance
    # def getDiscountType(row):
    #     if pd.isnull(row):
    #         return np.nan
    #     elif ':' in row:
    #         return 1
    #     else:
    #         return 0

    # def convertRate(row):
    #     """Convert discount to rate"""
    #     if pd.isnull(row):
    #         return 1.0
    #     elif ':' in str(row):
    #         rows = row.split(':')
    #         return 1.0 - float(rows[1])/float(rows[0])
    #     else:
    #         return float(row)

    # def getDiscountMan(row):
    #     if ':' in str(row):
    #         rows = row.split(':')
    #         return int(rows[0])
    #     else:
    #         return 0

    # def getDiscountJian(row):
    #     if ':' in str(row):
    #         rows = row.split(':')
    #         return int(rows[1])
    #     else:
    #         return 0
    # print("tool is ok.")

In [ ]:
# 1
    def processData(df):
        # convert discunt_rate
        df['discount_rate'] = df['Discount_rate'].apply(convertRate)
        df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
        df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
        df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
        print(df['discount_rate'].unique())
        # convert distance
        df['distance'] = df['Distance'].fillna(-1).astype(int)
        return df

    dfoff = processData(dfoff)
    dftest = processData(dftest)

    date_received = dfoff['Date_received'].unique()
    date_received = sorted(date_received[pd.notnull(date_received)])

    date_buy = dfoff['Date'].unique()
    date_buy = sorted(date_buy[pd.notnull(date_buy)])
    date_buy = sorted(dfoff[dfoff['Date'].notnull()]['Date'])
    couponbydate = dfoff[dfoff['Date_received'].notnull()][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
    couponbydate.columns = ['Date_received','count']
    buybydate = dfoff[(dfoff['Date'].notnull()) & (dfoff['Date_received'].notnull())][['Date_received', 'Date']].groupby(['Date_received'], as_index=False).count()
    buybydate.columns = ['Date_received','count']

    print("end")

In [ ]:
for i in dfoff.columns:
    if sum(dfoff[i].isnull())!=0: print(i)

Coupon_id
Discount_rate
Distance
Date_received
Date
u35
u36
u37
m0
c3
c12


In [ ]:
print(dfoff.Date_received.apply(type))
print(type(np.nan))

In [ ]:
def label(row):
    if pd.isnull(row['Date_received']):
        return -1
    if pd.notnull(row['Date']):
        td = row['Date'] - row['Date_received']
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
dfoff['label'] = dfoff.apply(label, axis = 1)
dfoff = get_offline_features(dfoff,dfoff)
print("end")

1754884 8
Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'u5'],
      dtype='object')
1754884 25
end


In [ ]:
# data split
print("-----data split------")
df = dfoff[dfoff['label'] != -1].copy()
train = df[(df['Date_received'] < date(2016,5,16))].copy()
valid = df[(df['Date_received'] >= date(2016,5,16)) & (df['Date_received'] <= date(2016,6,15))].copy()
print("end")

-----data split------
end


In [ ]:
# feature
original_feature = ['Distance', 'u5', 'u35', 'u36', 'u37',
       'discount_type', 'm0', 'm1', 'm5', 'm20', 'm21', 'c1',
       'c2', 'c3', 'c6', 'c12', 'um4', 'o1']
train = train.fillna(-1)
print("----train-----")
model = SGDClassifier(#lambda:
    loss='log',
    penalty='elasticnet',
    fit_intercept=True,
    max_iter=100,
    shuffle=True,
    alpha = 0.01,
    l1_ratio = 0.01,
    n_jobs=1,
    class_weight=None
)
model.fit(train[original_feature], train['label'])

----train-----


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.01, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.01, learning_rate='optimal', loss='log', max_iter=100,
              n_iter_no_change=5, n_jobs=1, penalty='elasticnet', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
# test prediction for submission
test = dftest.fillna(-1)
y_test_pred = model.predict_proba(test[original_feature])
dftest1 = dftest[['User_id','Coupon_id','Date_received']].copy()
dftest1['label'] = y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head()

,User_id,Coupon_id,Date_received,label
0,4129537,9983,2016-07-12,1.505297e-04
1,6949378,3429,2016-07-06,5.011163e-01
2,2166529,6928,2016-07-27,3.295629e-17
3,2166529,1808,2016-07-27,2.441665e-16
4,6172162,6500,2016-07-08,3.061301e-07
